# Web Scraping Weather Data

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime


#Selects the date range for the web scraping, currently set to 2022 full year
start_date = datetime.datetime(2022, 1, 1)
today_date = datetime.datetime(2022, 12, 31)
# today_date = datetime.datetime.today()


# Generate a list of dates between the start and end dates
date_range = [start_date + datetime.timedelta(days=x) for x in range((today_date - start_date).days + 1)]

# Format the dates as strings delimited by hyphens for our link
date_range_hyphen = [date.strftime('%Y-%m-%d') for date in date_range]

for date in date_range_hyphen:
    # Specify the URL and use requests to get the HTML content
    url = f'https://www.wunderground.com/dashboard/pws/KNYJACKS2/table/{today_date}/{date}/daily'
    response = requests.get(url)
    html_content = response.content

    # Parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the last table on the page and extract the data into a list of lists
    print(date)
    table = soup.find_all('table')[-1]
    rows = table.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        data.append(cols)
    data[0] = ['Time', 'Temperature', 'Dew_Point', 'Humidity', 'Wind', 'Speed', 'Gust', 'Pressure', 'Precip_Rate', 'Precip_Accum', 'UV', 'Solar']

    # Convert the list of lists into a pandas dataframe
    # Print statements just show where data may have to be imputed, and also slow the program down so it doesn't outpace my internet
    try:
        df = pd.DataFrame(data[2:], columns=data[0])
    except:
        print(date)
    df['Time'] = pd.to_datetime(df['Time'], format='%I:%M %p')
    df['hour'] = df['Time'].dt.hour
    try:
        df.drop('Wind', axis = 1, inplace = True)
    except:
        print(date, 'wind problem')

    #Tries to change datatypes to numerical data, print statements show where data will need to be imputed later
    for col in df:
        if col == 'UV':
            df[col] = pd.to_numeric(df[col])
        elif df[col].dtypes == 'object':
            df[col] = df[col].map(lambda x: x.split()[0])
            try:
                df[col] = pd.to_numeric(df[col])
            except:
                print(date, col)
    df_hourly = df.groupby(df.hour).mean()
    df_hourly['date'] = date
    df_hourly.reset_index(inplace=True)

    #Inits our weather dataframe the first pass through, and concats grouped data to it each time after, reusing as much as possible to use less ram
    if date == str(start_date.date()):
        dfw = df_hourly
    else:
        dfw = pd.concat([dfw,df_hourly], ignore_index=True)

2022-01-01
2022-01-02
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-10 Gust
2022-01-11
2022-01-12
2022-01-12 Gust
2022-01-13
2022-01-14
2022-01-15
2022-01-16
2022-01-16 Temperature
2022-01-16 Dew_Point
2022-01-16 Humidity
2022-01-16 Speed
2022-01-16 Gust
2022-01-16 Solar
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-25
2022-01-25 Gust
2022-01-26
2022-01-27
2022-01-28
2022-01-28 Gust
2022-01-29
2022-01-30
2022-01-30 Gust
2022-01-31
2022-01-31 Gust
2022-02-01
2022-02-01 Gust
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-09 Gust
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16
2022-02-17
2022-02-18
2022-02-19
2022-02-20
2022-02-21
2022-02-22
2022-02-23
2022-02-24
2022-02-25
2022-02-26
2022-02-26 Gust
2022-02-27
2022-02-28
2022-03-01
2022-03-02
2022-03-03
2022-03-04
2022-03-04 Gust
2022-03-05
2022-03-06
2022-03-07


## Reading in Taxi Data

In [5]:
#List of months to iterate through the files
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11','12']

In [16]:
#Creates loop to read in and concat 
i=1
for month in months:                   
    df = pd.read_parquet(f'yellow_tripdata_{year}-{month}.parquet', engine='pyarrow')

    df['date'] = pd.to_datetime(df['tpep_pickup_datetime'].dt.date)
    df['hour'] = df['tpep_pickup_datetime'].dt.hour

    df['Trip_Length'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    if month != months[-1]:
        start = pd.to_datetime(f'{year}-{month}-01')
        end = pd.to_datetime(f'{year}-{months[i]}-01')
        df = df[(df['date'] >= start) & (df['date'] < end)]
        i +=1
    else:
        df = df[(df['date'] >= pd.to_datetime(f'{year}-{month}-01')) & (df['date'] < pd.to_datetime(f'2023-01-01'))]
    df = df.groupby(['date', 'hour']).agg({'passenger_count':['sum', 'count'], 'trip_distance':['sum', 'mean'], 'total_amount':'sum', 'Trip_Length':['sum', 'mean']})
    df.reset_index(inplace=True)
    if month == '01':
        dfg = df
    else:
        dfg = pd.concat([dfg,df], ignore_index = True)


In [31]:
dfw.date.value_counts()

2022-01-01    24
2022-09-07    24
2022-09-05    24
2022-09-04    24
2022-09-03    24
              ..
2022-03-13    23
2022-11-27    21
2022-06-01    17
2022-09-21    12
2022-06-02    12
Name: date, Length: 363, dtype: int64

### Merging the data

In [23]:
# Resets column names for clarity
dfg.columns = ['date', 'hour', 'Total Passengers', 'Amount of Trips', 'Total Distance', 'Average Distance','Total Fares', 'Total Time', 'Averge Time']

In [34]:
#Changes datatype for merge
dfw['date'] = pd.to_datetime(dfw['date'])

In [36]:
#Inner Join the two data sets
dfgw = dfg.set_index(['date', 'hour']).join(dfw.set_index(['date', 'hour']), how='inner')

Send to CSV for EDA

In [38]:
dfgw.to_csv('TaxiAndWeather.csv')